In [3]:
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [4]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations for data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize pixel values
])

# Download and load the CIFAR-10 training data
trainset = datasets.CIFAR10(root='./data', download=True, train=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)


0.1%


KeyboardInterrupt: 

In [26]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),   # Randomly flip images horizontally
    transforms.RandomCrop(32, padding=4),# Randomly crop with padding
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize
])

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 256)
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1) # Flatten the image
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10): # 10 epochs
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()       # Zero the gradients
        
        outputs = model(images)     # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward()             # Backward pass
        optimizer.step()            # Update weights
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 1.884612083282617
Epoch 2, Loss: 1.6646840758335866
Epoch 3, Loss: 1.5801173422647559
Epoch 4, Loss: 1.5265359328226056
Epoch 5, Loss: 1.4823192079048937
Epoch 6, Loss: 1.444394311331727
Epoch 7, Loss: 1.410065675635472
Epoch 8, Loss: 1.3786760086903487
Epoch 9, Loss: 1.349889899687389
Epoch 10, Loss: 1.3214048921604595


In [28]:
# Validation data transformations (only normalization, no augmentation)
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values
])

# Download and load the CIFAR-10 validation data
valset = datasets.CIFAR10(root='./data', download=True, train=False, transform=val_transform)
valloader = DataLoader(valset, batch_size=64, shuffle=False)


Files already downloaded and verified


In [29]:
# Validation loop
val_loss = 0.0
correct = 0
total = 0

# Switch the model to evaluation mode
model.eval()

with torch.no_grad():  # No need to track gradients during validation
    for images, labels in valloader:
        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute the loss on the validation set
        val_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get the predicted class with the highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
avg_val_loss = val_loss / len(valloader)
accuracy = 100 * correct / total

print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}%")


Validation Loss: 1.4007310176351269, Validation Accuracy: 50.65%


In [30]:
import torch.nn.functional as F

class SimpleNNWithDropout(nn.Module):
    def __init__(self):
        super(SimpleNNWithDropout, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 256)
        self.dropout = nn.Dropout(0.5)  # Drop 50% of neurons
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        return self.fc2(x)


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim



# Initialize the model, loss function, and optimizer
model = SimpleNNWithDropout()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10): # 10 epochs
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()       # Zero the gradients
        
        outputs = model(images)     # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward()             # Backward pass
        optimizer.step()            # Update weights
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 1.9193771919021216
Epoch 2, Loss: 1.7332191090754536
Epoch 3, Loss: 1.668132391427179
Epoch 4, Loss: 1.6252766742425806
Epoch 5, Loss: 1.5915023487852054
Epoch 6, Loss: 1.5667871852664996
Epoch 7, Loss: 1.5418601470530187
Epoch 8, Loss: 1.5187764446753675
Epoch 9, Loss: 1.505373979011155
Epoch 10, Loss: 1.4896475395278248


In [32]:
# Validation loop
val_loss = 0.0
correct = 0
total = 0

# Switch the model to evaluation mode
model.eval()

with torch.no_grad():  # No need to track gradients during validation
    for images, labels in valloader:
        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute the loss on the validation set
        val_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get the predicted class with the highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
avg_val_loss = val_loss / len(valloader)
accuracy = 100 * correct / total

print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}%")

Validation Loss: 1.443209635224312, Validation Accuracy: 50.16%


In [33]:
class SimpleNNWithBatchNorm(nn.Module):
    def __init__(self):
        super(SimpleNNWithBatchNorm, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 256)
        self.batch_norm = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.batch_norm(self.fc1(x)))  # Apply batch normalization
        return self.fc2(x)


In [34]:
import torch
import torch.nn as nn
import torch.optim as optim



# Initialize the model, loss function, and optimizer
model = SimpleNNWithBatchNorm()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10): # 10 epochs
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()       # Zero the gradients
        
        outputs = model(images)     # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward()             # Backward pass
        optimizer.step()            # Update weights
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 1.7124282196354683
Epoch 2, Loss: 1.5277353299548253
Epoch 3, Loss: 1.4471444903737138
Epoch 4, Loss: 1.3867907578987844
Epoch 5, Loss: 1.3399442193453268
Epoch 6, Loss: 1.2988353524061724
Epoch 7, Loss: 1.2623958051814448
Epoch 8, Loss: 1.2273711007269448
Epoch 9, Loss: 1.1980362065765253
Epoch 10, Loss: 1.16894822779214


In [35]:
# Validation loop
val_loss = 0.0
correct = 0
total = 0

# Switch the model to evaluation mode
model.eval()

with torch.no_grad():  # No need to track gradients during validation
    for images, labels in valloader:
        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute the loss on the validation set
        val_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get the predicted class with the highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
avg_val_loss = val_loss / len(valloader)
accuracy = 100 * correct / total

print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}%")

Validation Loss: 1.3969063143821279, Validation Accuracy: 50.38%
